## 目的：模型确定， 对比   （ 直接drop非数值列  、 one-hot处理非数值列） 

## 过程：
1.drop 和 one-hot

2.missing data 处理

In [52]:
import pandas as pd
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [58]:
predictors = train_data.loc[:, train_data.columns != 'SalePrice']
target = train_data.SalePrice

### drop 和 one-hot


In [59]:
# Drop those non_numeric columns
numeric_predictors = predictors.select_dtypes(exclude=["object"])

# One-hot
one_hot_predictors = pd.get_dummies(predictors)

### missing data 处理

In [60]:
# numeric_predictors
from sklearn.preprocessing import Imputer
my_imputer = Imputer()
cols_with_missing = (col for col in numeric_predictors.columns if numeric_predictors[col].isnull().any())
for col in cols_with_missing:
    numeric_predictors[col+'_was_missing'] = numeric_predictors[col].isnull()
numeric_predictors = my_imputer.fit_transform(numeric_predictors)

C:\Users\JialiangShi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [61]:
# one_hot_predictors
cols_with_missing = (col for col in one_hot_predictors.columns if one_hot_predictors[col].isnull().any())
for col in cols_with_missing:
    one_hot_predictors[col+'_was_missing'] = one_hot_predictors[col].isnull()
one_hot_predictors = my_imputer.fit_transform(one_hot_predictors)

### Cross Validation

In [81]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

def get_mae(X, y):
    # multiple by -1 to make positive MAE score instead of neg value returned as sklearn convention
    return -1 * cross_val_score(RandomForestRegressor(50), 
                                X, y, 
                                scoring = 'neg_mean_absolute_error').mean()

In [82]:
get_mae(numeric_predictors,target)

18676.380273813247

In [83]:
get_mae(one_hot_predictors,target)

18451.060289051693